In [5]:
import torch
import torchvision
from torchvision import transforms

import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import numpy as np

In [6]:
# Pytorch에서 제공하는 MNIST 데이터 로드
dataset = torchvision.datasets.MNIST("./data/", download = True, train = True,
                                     transform = transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 50, shuffle = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw



In [7]:
# 연산에 사용할 GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32,10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [9]:
# 모델 선언, 학습률과 손실함수, 최적화 방식 지정
learning_rate = 1e-4

model = AutoEncoder().to(device)
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
# 학습
for epoch in range(20):
    running_loss = 0.0
    for data in trainloader:
        inputs = data[0].to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs.view(-1, 28 * 28))
        outputs = outputs.view(-1, 1, 28, 28)
        
        loss = loss_func(inputs, outputs)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    cost = running_loss / len(trainloader)
    print(f"{epoch + 1} loss : {cost}")

1 loss : 0.08015835721045733
2 loss : 0.05806422209677597
3 loss : 0.04824936956788103
4 loss : 0.04343232287404438
5 loss : 0.039659225420715906
6 loss : 0.035942102681534985
7 loss : 0.03384630586951971
8 loss : 0.03255132369852314
9 loss : 0.03150119544782986
10 loss : 0.030428691434984407
11 loss : 0.02946227272041142
12 loss : 0.028807604447938503
13 loss : 0.0283108848053962
14 loss : 0.027882350290504596
15 loss : 0.02755028231224666
16 loss : 0.027237884999873736
17 loss : 0.026943353953150413
18 loss : 0.02666318786640962
19 loss : 0.026411988770899675
20 loss : 0.026191519433632494
